In [4]:
import pandas as pd
from texas_hold_em_utils.card import Card
from texas_hold_em_utils.deck import Deck
from texas_hold_em_utils.game import Game
from texas_hold_em_utils.hands import HandOfTwo

In [1]:
# for a given pair of hole cards and a given flop, what is that hand's probability of winning?
# too many cards to break it down to suited/offsuite, so there are 52 first cards, 51 second cards, 50 third cards, 49 fourth cards, 48 fifth cards... 311 million combinations
52*51*50*49*48

# maybe go with sample win rates instead

311875200

In [20]:
def run_game(hole_cards, flop, player_count):
    game1 = Game(player_count)
    deck = Deck()
    deck.remove(hole_cards[0])
    deck.remove(hole_cards[1])
    deck.remove(flop[0])
    deck.remove(flop[1])
    deck.remove(flop[2])
    game1.deck = deck
    game1.community_cards = flop
    game1.players[0].hand_of_two = HandOfTwo([hole_cards[0], hole_cards[1]])
    deck.shuffle()
    for i in range(1, player_count):
        game1.players[i].hand_of_two = HandOfTwo([deck.draw(), deck.draw()])
        
    game1.round = 1
    game1.turn()
    game1.river()
    
    winners = game1.determine_round_winners()
    if 0 in [winner.position for winner in winners]:
        return 1 / len(winners)
    return 0
    

In [27]:
# win rates seem low, and vary a lot, need to test/debug
hole_cards = [Card().from_str('A', 'Hearts'), Card().from_str('A', 'Spades')]
flop = [Card().from_str('4', 'Diamonds'), Card().from_str('6', 'Clubs'), Card().from_str('8', 'Spades')]

win_ct = 0.0
for i in range(60):
    win_ct += run_game(hole_cards, flop, 3)
    
print(win_ct / 60)

0.15013045353826512
